In [1]:
import pandas as pd
import numpy as np

Retreive labeled test set by generating syntetic annotations based on the confidence of the lables

In [2]:
test_set = pd.read_csv("../../Data/resultsNNEnglish.csv")

FileNotFoundError: [Errno 2] No such file or directory: '../../Data/resultsNNEnglish.csv'

In [ ]:
test_set = test_set[["iaa","Truth","Pred"]].sample(2918)

In [ ]:
def simulation_function(row):
  rand = np.random.uniform()
  return row["iaa"] > rand

In [ ]:
test_set["isCorrectAnno"] = test_set.apply(simulation_function, axis=1)

In [ ]:
test_set

,iaa,Truth,Pred,isCorrectAnno
1074,1.0000,False,True,True
4599,1.0000,False,False,True
1692,1.0000,False,False,True
2903,1.0000,False,False,True
4987,0.4140,True,True,False
...,...,...,...,...
849,0.6907,True,False,False
5632,0.6290,True,True,True
4217,0.7027,True,True,True
1214,1.0000,False,True,True


In [ ]:
def task_routing_func(row):
  if row["Pred"]:
    return True
  else:
    return row["isCorrectAnno"]

In [ ]:
test_set["isCorrectAnnoTaskRouting"] = test_set.apply(task_routing_func,axis=1)

In [ ]:
num_tru_pred = test_set["Pred"].sum()

In [ ]:
random_selected = test_set.sample(num_tru_pred).index

In [ ]:
def random_routing_func(row):
  if row["randomSelected"]:
    return True
  else:
    return row["isCorrectAnno"]

In [ ]:
test_set["randomSelected"] = test_set.index.isin(random_selected)

In [ ]:
test_set["isCorrectAnnoRandom"] = test_set.apply(random_routing_func, axis=1)

In [ ]:
one_dataset_cost = len(test_set)

In [ ]:
one_dataset_acc = test_set["isCorrectAnno"].sum()/len(test_set)

In [ ]:
thesis_dataset_cost = 5*test_set["Pred"].sum() + (len(test_set) - test_set["Pred"].sum())

In [ ]:
thesis_dataset_acc = test_set["isCorrectAnnoTaskRouting"].sum()/len(test_set)

In [ ]:
rand_data_acc = test_set["isCorrectAnnoRandom"].sum()/len(test_set)

In [ ]:
gr_dataset_cost = len(test_set)*5

Cost and accuracy calculations

In [ ]:
from sklearn.preprocessing import scale

In [ ]:
name = ["One Annotator", "Random Model with Task Routing", "Task Routing", "Golden Rule"]
accuracy = [round(one_dataset_acc,2),round(rand_data_acc,2), round(thesis_dataset_acc,2),1]
cost = [one_dataset_cost/gr_dataset_cost, thesis_dataset_cost/gr_dataset_cost,thesis_dataset_cost/gr_dataset_cost ,gr_dataset_cost/gr_dataset_cost]
cost_anno = [one_dataset_cost, thesis_dataset_cost ,thesis_dataset_cost,gr_dataset_cost]
is_random_model = [False,True,False,False]

In [ ]:
import plotly.graph_objects as go

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Bar(name='Accuracy',
           x=name,
           y=accuracy,
           text=accuracy,
           textposition='auto'))

fig.add_trace(
    go.Bar(name='Cost',
           x=name,
           y=cost,
           text=cost_anno,
           textposition='auto'))

fig.update_layout(
    template="plotly_white",
    title_text='Cost and Accuracy Comparison English',
    title_font_size=30,
    legend = dict(font = dict(size = 18)),
    width=1200, height=600,
    font=dict(
        family="Helvetica",
    )
)
fig.update_xaxes(title_font=dict(size=22))
fig.update_yaxes(title_font=dict(size=22))
fig.show()

In [ ]:
name = ["One Annotator", "Task Routing (1 or 5 annotators)", "Golden Rule (5 annotators)"]
accuracy = [round(one_dataset_acc,2), round(thesis_dataset_acc,2),1]
cost = [one_dataset_cost/gr_dataset_cost,thesis_dataset_cost/gr_dataset_cost ,gr_dataset_cost/gr_dataset_cost]
cost_anno = [one_dataset_cost, thesis_dataset_cost,thesis_dataset_cost,gr_dataset_cost]

rand_acc = round(rand_data_acc,2)
rand_cost = thesis_dataset_cost/gr_dataset_cost
rand_name = "Random Model with Task Routing (1 or 5 annotators)"

In [ ]:
import plotly.graph_objects as go

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
           x=cost,
           y=accuracy,
           text=name,
           mode="lines+markers+text",
           textposition="top center",
           showlegend=False
           ))
fig.add_trace(
    go.Scatter(
           x=[rand_cost],
           y=[rand_acc],
           mode='markers+text',
           text=rand_name,
           textposition="bottom center",
           showlegend=False
           ))

fig.update_layout(
    template="plotly_white",
    title_text='Cost and Accuracy Comparison English',
    title_font_size=30,
    legend = dict(font = dict(size = 18)),
    width=1200, height=600,
    font=dict(
        family="Helvetica",
    ),
    xaxis_title="Cost",
    yaxis_title="Accuracy",
)
fig.update_traces(marker=dict(size=12))